# CelebA

CelebA 数据集非常适合训练和测试用于面部检测的模型，尤其适用于识别面部特征，例如找到棕色头发的人，微笑着或戴眼镜的人。图像涵盖大的姿势变化，背景混乱，各种各样的人，并得到大量图像和丰富的标注的支持。这些数据最初是由香港中文大学 MMLAB 的研究人员收集的。

概览：

- 202,599名人的面部图像数量
- 10,177个唯一身份，但未提供身份名称
- 每个图像 40 个二进制属性注释
- 5个地标性位置（landmark locations）

数据文件：

- `imgalignceleba.zip`：裁剪和对齐的所有面部图像
- `listevalpartition.csv`：建议将图像划分为训练，验证和测试集。图像 1-162770 是训练集，图像 162771-182637 是验证集，182638-202599 是测试集。
- `listbboxceleba.csv`：每个图像的边界框信息。"x1"  和 "y1" 表示边界框的左上点坐标。"width"  和  "height" 代表边界框的宽度和高度。
- `listlandmarksalign_celeba.csv`：图像 landmarks 及其各自的坐标。有 5 个 landmarks：左眼，右眼，鼻子，左嘴，右嘴。
- `listattrceleba.csv`：每个图像的属性标签。有 40 个属性。 "1"  代表正，而  "-1"  代表负。

原始数据和 banner 图像来源来自 <http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html>。如网站上所述，CelebA 数据集仅可用于非商业研究目的。该数据集的创建者撰写了以下论文，这些论文使用CelebA进行人脸检测：

>S. Yang, P. Luo, C. C. Loy, and X. Tang, "From Facial Parts Responses to Face Detection: A Deep Learning Approach", in IEEE International Conference on Computer Vision (ICCV), 2015

配置深度学习环境：

```sh
$ conda install pandas matplotlib tqdm
$ pip install tensorflow
```

GPU 版的安装：

```sh
$ conda install pandas matplotlib tqdm
$ conda install cudatoolkit=10.1
$ conda install cudnn=7.6.5
$ pip install tensorflow-gpu
```

测试是否成功安装：

```python
import tensorflow as tf

A = tf.constant([[1, 2], [3, 4]])
B = tf.constant([[5, 6], [7, 8]])
C = tf.matmul(A, B)
tf.print(C)
```

输出：

```js
[[19 22]
 [43 50]]
```

## 1 数据解析

识别各种应用程序的脸部，从脸部登录手机或在监视图像中搜索特定的嫌疑人是计算机视觉和深度学习的一个相当受欢迎的研究领域。

从 Kaggle 下载数据：[CelebFaces Attributes (CelebA)](https://www.kaggle.com/jessicali9530/celeba-dataset) 并解压：

```python
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

PIC_DIR = 'img_align_celeba/img_align_celeba/'
ROOT = Path(PIC_DIR)
len(list(ROOT.iterdir())) # 查看数据量
```

输出：

```js
202599
```

统计图像的尺寸：

```python
sizes = set()
for name in ROOT.iterdir():
    sizes.add(Image.open(name).size)

sizes # 输出全部尺寸
```

输出：

```js
{(178, 218)}
```

使用 TensorFLow 读取图片：

```python
# 读取图片到 tf
from matplotlib import pyplot as plt
WIDTH = 128
HEIGHT = 128

ds = tf.data.Dataset.list_files(PIC_DIR+"*.jpg")


@tf.function
def load_image(img_path, size=(WIDTH, HEIGHT)):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img)  # 注意此处为jpeg格式
    img = tf.image.resize(img, size)
    return img

# 可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.figure(1, figsize=(10, 10))
for i, img in enumerate(ds.map(load_image).take(25)):
    plt.subplot(5, 5, i+1)
    plt.imshow(img/255)
    plt.axis('off')
plt.show()
```

显示：

![](https://upload-images.jianshu.io/upload_images/1114626-b78890b0e887c322.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

## 2 GAN

生成对抗网络（GAN）是一种使用两个神经网络的算法体系结构，将一个神经网络与另一个神经网络（因此称为“对抗性”）对决，以便生成可以传递给真实数据的新的合成数据实例。它们被广泛用于图像生成，视频生成和语音生成。

GAN 由两部分组成：

- 生成器（Generator）：生成新的数据实例
- 鉴别器（Discriminator）：尝试从真实数据集中区分生成的数据或假数据（fake data）。

判别算法尝试对输入数据进行分类；也就是说，给定数据实例的特征，它们可以预测该数据所属的标签或类别。因此，判别算法将特征映射到标签。他们只关心这种相关性。另一种松散地说，生成算法则相反。他们尝试预测给定特定标签的特征，而不是预测给定特定特征的标签。

在训练过程中，它们都从头开始，并且生成器通过训练时期学习塑造随机分布。

![](https://upload-images.jianshu.io/upload_images/1114626-ac94150fcace1b96.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

### 2.1 构建模型
